README: Just run the code blocks sequentially to see the magic. Comments are also there to help you.

In [1]:
import z3
from time import time

grid = [
    [z3.Int('g{}{}'.format(i, j)) for i in range(9)] 
    for j in range(9)
]

# first we represent the usual sudoku rules as z3 constraints 
sudoku_constraints = []
for cell1 in range(81):
    r1 = cell1 // 9
    c1 = cell1 % 9
    # each cell must have a number in [1..9]
    sudoku_constraints.append(grid[r1][c1] >= 1)
    sudoku_constraints.append(grid[r1][c1] <= 9)
    box1 = (3 * (r1 // 3), 3 * (c1 // 3))  
    # box1 represents the 3x3 box the cell belongs to
    for cell2 in range(cell1+1, 81):
        r2 = cell2 // 9
        c2 = cell2 % 9
        box2 = (3 * (r2 // 3), 3 * (c2 // 3))
        if box1 == box2 or r1 == r2 or c1 == c2:
            sudoku_constraints.append(grid[r1][c1] != grid[r2][c2])
            # if the two cells are in the same box,
            # the same row, or the same column,
            # then they must not be equal.


In [2]:
# get input from a text file
with open('sudoku.txt') as file:
    input_grid = [list(map(int, list(line[:-1]))) for line in file]

# manual input (same as that got from text file)
input_grid = [[0, 0, 8, 5, 0, 0, 0, 4, 0], 
              [0, 6, 0, 0, 0, 0, 0, 7, 3], 
              [0, 0, 4, 0, 0, 0, 0, 9, 2], 
              [0, 0, 9, 0, 0, 1, 0, 0, 0], 
              [0, 0, 7, 0, 2, 0, 0, 1, 9], 
              [8, 0, 0, 0, 9, 0, 0, 0, 0], 
              [5, 0, 0, 6, 3, 0, 0, 0, 0], 
              [0, 0, 0, 0, 0, 4, 0, 3, 0], 
              [1, 4, 0, 0, 0, 0, 0, 0, 0]]


def pretty_print(grid, m=None):
    print('+-------'*3 + '+')
    for k in range(3):
        for i in range(3*k, 3*k+3):
            for l in range(3):
                print('|', end=' ')
                for j in range(3*l, 3*l+3):
                    if m:
                        print(m[grid[i][j]], end=' ')
                    else:
                        print(
                            grid[i][j] if grid[i][j] else '_', 
                            end=' '
                        )
            print('|')
        print('+-------'*3 + '+')
    return None


pretty_print(input_grid)

+-------+-------+-------+
| _ _ 8 | 5 _ _ | _ 4 _ |
| _ 6 _ | _ _ _ | _ 7 3 |
| _ _ 4 | _ _ _ | _ 9 2 |
+-------+-------+-------+
| _ _ 9 | _ _ 1 | _ _ _ |
| _ _ 7 | _ 2 _ | _ 1 9 |
| 8 _ _ | _ 9 _ | _ _ _ |
+-------+-------+-------+
| 5 _ _ | 6 3 _ | _ _ _ |
| _ _ _ | _ _ 4 | _ 3 _ |
| 1 4 _ | _ _ _ | _ _ _ |
+-------+-------+-------+


In [3]:
# the numbers already present in the input must match
input_constraints = []
for i in range(9):
    for j in range(9):
        if input_grid[i][j] != 0:
            input_constraints.append(grid[i][j] == input_grid[i][j])

In [4]:
# and now we solve
solver = z3.Solver()
solver.add(sudoku_constraints)  # the general sudoku rules
solver.add(input_constraints)  # the grid agrees with the input grid

t1 = time()
res = solver.check()
tf = time()
if res == z3.sat:
    m = solver.model()
    pretty_print(grid, m)
else:
    print('No solution for this sudoku. :(')
    
print(f'Time taken by z3: {tf-t1} secs')

+-------+-------+-------+
| 3 9 8 | 5 7 2 | 6 4 1 |
| 2 6 1 | 9 4 8 | 5 7 3 |
| 7 5 4 | 3 1 6 | 8 9 2 |
+-------+-------+-------+
| 4 2 9 | 7 6 1 | 3 5 8 |
| 6 3 7 | 8 2 5 | 4 1 9 |
| 8 1 5 | 4 9 3 | 7 2 6 |
+-------+-------+-------+
| 5 7 2 | 6 3 9 | 1 8 4 |
| 9 8 6 | 1 5 4 | 2 3 7 |
| 1 4 3 | 2 8 7 | 9 6 5 |
+-------+-------+-------+
Time taken by z3: 0.07476019859313965 secs
